In [24]:
import pandas as pd 
import zipfile
import requests
from datetime import datetime
import time
import json

classes = pd.read_csv('elliptic_bitcoin_dataset/elliptic_txs_classes.csv')
deanonym = pd.read_csv("elliptic_bitcoin_dataset/deanonymized_result.csv")

df=classes.merge(deanonym, on=["txId"])

labeled_txs = df
labeled_txs.loc[labeled_txs["class"]=="unknown", "class"] = "3"
labeled_txs.head()

,txId,class,transaction
0,230425980,3,74d9bb85c6bbc471c6e18f409d23c3ef1191725bdb9037...
1,5530458,3,906c816344eb837a6ddcf75dece1c07f0c2a87885e0ea9...
2,232022460,3,6e01a383ce4d9e2ca6b3eb0725d0775fb83be4b91283cd...
3,232438397,2,1c61975e191e5713a0a3db62eb352314592585b18c18ee...
4,230460314,3,996c249ed096dbbb208f8c5247a09b0fe8e740d99189e3...


In [53]:
good_txs = labeled_txs[labeled_txs['class'] == "2"]
bad_txs = labeled_txs[labeled_txs['class'] == "1"]
unknown_txs = labeled_txs[labeled_txs['class'] == "3"]

print(f"good_tx_list length: {len(good_txs)}")
print(f"bad_tx_list length: {len(bad_txs)}")
print(f"unknown_txs_list length: {len(unknown_txs)}")
print(f"Total length: {len(labeled_txs)}")

good_tx_list length: 10082
bad_tx_list length: 1726
unknown_txs_list length: 38893
Total length: 50701


In [54]:
json_list = []
txs_failed_list = []
# legal transactions
for i in range(len(good_txs)):
    try:
        response = requests.get(f"https://blockchain.info/rawtx/{good_txs['transaction'].iloc[i]}")
        tx_json = response.json()
        tx_json["elliptic_label"] = good_txs['class'].iloc[i]
        json_list.append(tx_json)
    except Exception:
        txs_failed_list.append(good_txs['transaction'].iloc[i])

# illegal transactions
for i in range(len(bad_txs)):
    try:
        response = requests.get(f"https://blockchain.info/rawtx/{bad_txs['transaction'].iloc[i]}")
        tx_json = response.json()
        tx_json["elliptic_label"] = bad_txs['class'].iloc[i]
        json_list.append(tx_json)
    except Exception:
        txs_failed_list.append(bad_txs['transaction'].iloc[i])
    
with open("elliptic_txs_json.json", "w") as f:
    f.writelines(json.dumps(json_list))
    

with open("failed_txs.json", "w") as f:
    f.writelines(txs_failed_list)